In [ ]:
import sqlite3

conn = sqlite3.connect("metadata.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS docs (
    doc_id INTEGER PRIMARY KEY AUTOINCREMENT,
    doc_name TEXT,
    
    status TEXT
)
""")